<a href="https://colab.research.google.com/github/mett29/DL-Competition/blob/master/image_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

# This permits to load the dataset on colab.
# We chose to take advantage of github to rapidly load all the data on colab.

from getpass import getpass
import os

user = getpass('Github user')
password = getpass('Github password')
os.environ['GITHUB_AUTH'] = user + ':' + password

!git clone https://$GITHUB_AUTH@github.com/DanieleParravicini/DL-CompetitionsDatasets
os.environ['GITHUB_AUTH'] = ''

# Of course it is not mandatory to load the dataset from a remote repo.
# You can simply unpack the kaggle competition zip in a local directory named 'DL-CompetitionsDatasets'
# As depicted below:

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

import time
import os
import numpy as np
import matplotlib.pyplot as plt

2.0.0


# Loading datasets

In [0]:
SEED = 2019
tf.random.set_seed(SEED)

In [0]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
PATH = ''
on_kaggle = True
if on_kaggle: 
  PATH = "/kaggle/input/ann-and-dl-image-segmentation/Segmentation_Dataset"
else:
  PATH = 'DL-CompetitionsDatasets/Segmentation_Dataset'
epochs = 5
batch_size = 32

In [0]:
apply_data_augmentation = False

# We need two different generators for images and corresponding masks
if apply_data_augmentation:
    train_img_data_gen = ImageDataGenerator(rotation_range=10,
                                        zoom_range=[0.5,1],
                                        horizontal_flip=True,
                                        vertical_flip=True,  
                                        fill_mode='nearest',                           
                                        rescale=1./255,
                                        validation_split=0.2)
    train_mask_data_gen = ImageDataGenerator(rotation_range=10,
                                        zoom_range=[0.5,1],
                                        horizontal_flip=True,
                                        vertical_flip=True,  
                                        fill_mode='nearest',                           
                                        rescale=1./255,
                                        validation_split=0.2) 
else:
    train_img_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_mask_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [0]:
dataset_dir = os.path.join(PATH, 'training')

train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(dataset_dir, 'images'),
                                               batch_size=batch_size,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH),
                                               class_mode=None,
                                               shuffle=True,
                                               subset='training',
                                               #interpolation='bilinear',
                                               seed=SEED)

train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(dataset_dir, 'masks'),
                                               batch_size=batch_size,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH),
                                               class_mode=None,
                                               shuffle=True,
                                               subset='training',
                                               #interpolation='bilinear',
                                               color_mode='grayscale',
                                               seed=SEED)

train_gen = zip(train_img_gen, train_mask_gen)

# Validation
valid_img_gen = train_img_data_gen.flow_from_directory(os.path.join(dataset_dir, 'images'),
                                               batch_size=batch_size, 
                                               target_size=(IMG_HEIGHT, IMG_WIDTH),
                                               class_mode=None,
                                               shuffle=False,
                                               subset='validation',
                                               #interpolation='bilinear',
                                               seed=SEED)

valid_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(dataset_dir, 'masks'),
                                               batch_size=batch_size, 
                                               target_size=(IMG_HEIGHT, IMG_WIDTH),
                                               class_mode=None,
                                               shuffle=False,
                                               subset='validation',
                                               #interpolation='bilinear',
                                               color_mode='grayscale',
                                               seed=SEED)

valid_gen = zip(valid_img_gen, valid_mask_gen)

Found 6118 images belonging to 1 classes.
Found 6118 images belonging to 1 classes.
Found 1529 images belonging to 1 classes.
Found 1529 images belonging to 1 classes.


In [0]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 4, figsize=(15, 15))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [0]:
def grayscale_to_colour(gray_img):
  
  black = [0, 0, 0] # Background
  white = [255, 255, 255] # Buildings

  colour_img = np.zeros([gray_img.shape[0], gray_img.shape[1], 3])
  return np.where(gray_img == 1, white, black )


sample_training_images, sample_training_masks = next(train_gen)
plotImages(sample_training_images[:4])
sample_training_masks_imaged = list(map( grayscale_to_colour ,sample_training_masks[:4]))
plotImages(sample_training_masks_imaged)

In [0]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_HEIGHT, IMG_WIDTH, 3], [None, IMG_HEIGHT, IMG_WIDTH, 1]))

'''When using data augmentation on masks it's a good idea to cast mask tensor to tf.int32. 
When applying geometric transformations, like rotation or zoom, the output is interpolated so you will end up with values in [0, 1] 
which are no longer binary. Casting to integer will allow to obtain again binary masks.'''

def prepare_target(x_, y_):
    y2_ = tf.cast(tf.cast(y_, tf.int32), tf.float32)
    return x_, y2_

train_dataset = train_dataset.map(prepare_target)
train_dataset = train_dataset.repeat()
train_dataset.cache()

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_HEIGHT, IMG_WIDTH, 3], [None, IMG_HEIGHT, IMG_WIDTH, 1]))

valid_dataset = valid_dataset.map(prepare_target)
valid_dataset = valid_dataset.repeat()
valid_dataset.cache()

<DatasetV1Adapter shapes: ((None, 256, 256, 3), (None, 256, 256, 1)), types: (tf.float32, tf.float32)>

In [0]:
fig, ax = plt.subplots(1, 2)
fig.show()

colors_dict = {}
colors_dict[0] = [0, 0, 0] # Background
colors_dict[1] = [255, 255, 255] # Buildings

iterator = iter(train_dataset)

for _ in range(1):
    augmented_img, target = next(iterator)
    augmented_img = augmented_img[0]
    augmented_img = augmented_img * 255
    
    ax[0].imshow(np.uint8(augmented_img))
    ax[1].imshow(np.uint8(grayscale_to_colour(target[0])))

# Model

In [0]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    d = tf.keras.layers.SpatialDropout2D(0.1)(c)
    p = tf.keras.layers.MaxPool2D((2, 2), (2, 2))(d)
    return d, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    #us = tf.keras.layers.UpSampling2D((2, 2))(x)
    us =  tf.keras.layers.Conv2DTranspose(filters ,(2, 2), strides=(2,2))(x)
    concat = tf.keras.layers.Concatenate()([us, skip])
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [0]:
def UNet():
    # Instead of a single model return an array of them
    # All of them share the convolution part of the network and 
    # part of the upsampling
    
    models = []
    
    f = [32, 32, 64, 128, 256]
    inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, 3))
    p0 = inputs
    
    downsampling_layers  = [p0]
    convolutional_layers = []
    # create a downsampling part of the network
    # and save the convolutions to enable creation of skip connection
    for i, n_filt in enumerate(f[:-1]):
        cx, px = down_block(downsampling_layers[i], n_filt) # spatial extent 2A x 2A -> A x A
        downsampling_layers.append(px)
        convolutional_layers.append(cx)
    
    bn = bottleneck(downsampling_layers[-1], f[-1])
    
    #create the upsampling part of the network
    upsampling_layers = [bn]
    for i, n_filt in enumerate(reversed(f[:-1]) ):
        ux = up_block(upsampling_layers[i], convolutional_layers[- i -1], n_filt ) # spatial extent A x A -> 2A x 2A
        upsampling_layers.append(ux)

    
    # for each upsampling layer create a model that can be trained with the same couple of full resolution 
    # image/mask as the whole nn.
    # In order to do so we have to upsample accordingly
    for i in range(len(upsampling_layers)-1):
        #up sample 
        upsampling_factor = 2 ** ( len(upsampling_layers) - i -1)
        upx     = tf.keras.layers.UpSampling2D((upsampling_factor, upsampling_factor))(upsampling_layers[i])
        outputx = tf.keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(upx)
        nn_name = 'upsampled_'+str(upsampling_factor)
        models.append(tf.keras.models.Model(inputs, outputx, name=nn_name))

    #create the final model
    output = tf.keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(upsampling_layers[-1])
    models.append( tf.keras.models.Model(inputs, output, name = "true_model"))
    return models

In [0]:
def IoU(y_true, y_pred):
    # from probability to predicted class {0, 1}
    y_pred = tf.cast(y_pred > 0.5, tf.float32) # when using sigmoid. Use argmax for softmax
    # A and B
    intersection = tf.reduce_sum(y_true * y_pred)
    # A or B
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    # IoU
    return intersection / union

In [0]:
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

In [0]:
from keras import backend as K

def intersection_over_union_loss(y_true, y_pred, smooth=1):
    """
    intersection_over_union_loss = (|X & Y|)/ (|X|+ |Y|- |X & Y|)
         =  sum(|A*B|)/(sum(A)+sum(B) - SUM(A * B))
    """
    # from probability to predicted class {0, 1} to a more steep value
    #y_pred1= K.sigmoid(20*(y_pred-0.5))
    y_pred1 = y_pred
    
    # A and B
    intersection = K.sum(y_true * y_pred1, axis=-1)

    # A or B
    union = K.sum(y_true, axis=-1) + K.sum(y_pred1, axis=-1) - intersection
    #union += K.constant((0,1))
    #intersection += K.constant((0,1))
    IOU = (K.abs(intersection) / K.abs(union + 0.1)) 
    
    return -(IOU)
    #intersection = K.sum(K.abs(y_true * y_pred1), axis=-1)
    #return (2. * intersection + smooth)/ (K.sum(K.square(y_true),-1)+ K.sum(K.square(y_pred),-1)+1 )


intersection_over_union_loss(  tf.constant([[1.0, 1.0], [0  , 0]]),
            tf.constant([[0  , 0.6], [0.0, 0]]) 
         )

In [0]:
models = UNet()

from tensorflow.keras.utils import plot_model
#to visuallly check models
plot_model(models[0])

In [0]:
lr = 1e-4
sgd = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)

#compile
for i, m in enumerate(models):
    if(i == 0):
        m.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[IoU])
    else
       m.compile(optimizer=sgd, loss='binary_crossentropy', metrics=[IoU])
    #m.compile(optimizer=optimizer, loss=intersection_over_union_loss, metrics=[IoU])
    m.summary()

# Training

In [0]:
# 0's are more frequent in the output than 1's.
# In order to take into account that unbalance we try to reweight the loss function
#collect weights
ite = iter(train_dataset)
weights = [0,0]
for i in range(len(train_img_gen)):
    batch = next(ite)
    for s in batch:
        img = s[0]
        mask = s[1]
        weights[0] += np.where(mask == 0)[0].shape[0]
        weights[1] += np.where(mask == 1)[0].shape[0]
        
#weights
print(weights)
tot = weights[0]+weights[1]

weights[0] = tot/weights[0]
weights[1] = tot/weights[1]


In [0]:
weights

In [0]:
from datetime import datetime

callbacks = []

# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callback.EarlyStopping(monitor='val_loss', patience=6)
    callbacks.append(es_callback)

In [0]:
for m in models:
    m.fit(x=train_dataset,
          epochs=10,
          steps_per_epoch=len(train_img_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_img_gen), 
          callbacks=callbacks
          ,class_weight = weights)

models[-1].save_weights("UNet_" + str(datetime.now().strftime('%b%d_%H-%M')) + ".h5")

#Load model from weight file



In [0]:
from ipywidgets import FileUpload
upload = FileUpload()
upload

In [0]:
for i, meta in enumerate(upload.metadata): 
    print(meta['name']) 
    with open(meta['name'], "w+b") as file_writer: 
        file_writer.write(upload.data[i])

In [0]:
#now you can load weigths 
#model.load_weights

In [0]:
models[-1].evaluate(valid_dataset,
          steps=len(valid_img_gen))

models[-1].evaluate(train_dataset,
          steps=len(train_img_gen))

48/48 [==============================] - 8s 169ms/step - loss: 0.2753 - IoU: 0.6739


[0.27531634146968526, 0.6738562]

# Collect results

In [0]:
def prediction_to_mask(prediction):
  return tf.cast(prediction > 0.5, tf.float32)

#for TRAINING purposes
image_filenames = next(os.walk(PATH+'/test/images/img'))[2]

results = {}
for image_name in image_filenames:

    img = image.load_img(os.path.join(PATH+'/test/images/img', image_name), target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_tensor = image.img_to_array(img)                    
    img_tensor = np.expand_dims(img_tensor, axis=0)         
    img_tensor /= 255. 
    
    #take the name of the file disregarding of the extension 
    name_without_ext = image_name[:-4]
    #obtain the prediction from the model
    prediction = models[-1].predict(img_tensor)[0]
   
    #transform into a mask
    results[name_without_ext] = prediction_to_mask(prediction)


In [0]:
#results
a = np.array([[255,255,255],[0,0,0]])

np.where(a == (255,255,255), (1,1,1), (2,2,2))
 

In [0]:
def plotImageAndMask(img, mask=None, pred=None):
  red = (255, 0, 0)
  black = (0,0,0)
  white = (255,255,255)
  green = (0, 255,0)

  plt.figure(figsize=(15, 15))
  plt.imshow(img, 'gray', interpolation='none')
  if(mask is not None):
      transf_mask = np.where( mask == white , red , black )
      plt.imshow(transf_mask, interpolation='none', alpha=0.2)
  if(pred is not None):
      transf_pred = np.where( pred == white , green , black )
      plt.imshow(transf_pred, interpolation='none', alpha=0.2)
  plt.show()
  #in red what has to be captured
  #in green what is captured only by our model
  #in orange what is captured by both


In [0]:
for image_name in image_filenames[:15]:
  

  img = image.load_img(os.path.join(PATH+'/test/images/img', image_name), target_size=(IMG_HEIGHT, IMG_WIDTH))
  #mask1 = image.load_img(os.path.join(PATH+'/training/masks/img', image_name), target_size=(IMG_HEIGHT, IMG_WIDTH))
  img_tensor = image.img_to_array(img)                    
  img_tensor = np.expand_dims(img_tensor, axis=0)         
  img_tensor /= 255. 

  name_without_ext = image_name[:-4]
  pred = results[name_without_ext]

  mask2 = grayscale_to_colour(pred)
  #plotImageAndMask(img, image.img_to_array(mask1), mask2)
  plotImageAndMask(img, pred=mask2)

In [0]:
for image_name in image_filenames[:15]:
  

  img = image.load_img(os.path.join(PATH+'/training/images/img', image_name), target_size=(IMG_HEIGHT, IMG_WIDTH))
  mask1 = image.load_img(os.path.join(PATH+'/training/masks/img', image_name), target_size=(IMG_HEIGHT, IMG_WIDTH))
  img_tensor = image.img_to_array(img)                    
  img_tensor = np.expand_dims(img_tensor, axis=0)         
  img_tensor /= 255. 

  name_without_ext = image_name[:-4]
  pred = results[name_without_ext]

  mask2 = grayscale_to_colour(pred)
  plotImageAndMask(img, image.img_to_array(mask1), mask2)


In [0]:
def rle_encode(img):
    # Flatten column-wise
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

#for img_name in results[:1]:
#    print(rle_encode(np.array(results[img_name])))

In [0]:
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(csv_fname, 'w') as f:

        f.write('ImageId,EncodedPixels,Width,Height\n')

        for key, value in results.items():
            f.write(key + ',' + str(rle_encode(np.array(value))) + ',' + '256' + ',' + '256' + '\n')

In [0]:
create_csv(results)

In [0]:
!kaggle competitions submit -c ann-and-dl-image-segmentation -f results_Dec04_17-25-10.csv -m "Trained Unet with successive refinement"